In [1]:
import pandas as pd

In [2]:
morula = pd.read_csv("morula_labels_cleaned.csv")
morula.head(2)

image                                         raw_output grade   stage
0  00997.png  <grade_description>\nFor the morula stage:\n\n...     C  morula
1  00977.png  <grade_description>\nFor the morula stage:\n\n...     A  morula

In [7]:
morula_images = morula.image.to_list()

In [3]:
morula.grade.value_counts()

B    767
A    164
C     83
Name: grade, dtype: int64

In [4]:
morula.stage.value_counts()

morula    1014
Name: stage, dtype: int64

In [10]:
import os

all_files = os.listdir("./morula/")
set(morula_images) - set(all_files)

set()

In [12]:
morula_images_new = ["morula" + file for file in morula_images]

In [13]:
for current_name, new_name in zip(morula_images, morula_images_new):
    os.rename(f"./morula/{current_name}", f"./morula/{new_name}")

In [48]:
morula['image'] = morula_images_new

In [14]:
eit_cell = pd.read_csv("8_cell_labels_cleaned.csv")
eit_cell.head(2)

image                                         raw_output grade   stage
0  00939.png  <stage_verification>\nThis appears to be an 8-...     A  8_cell
1  00949.png  <stage_verification>\nBased on the image, this...     A  8_cell

In [16]:
eit_cell_images = eit_cell.image.to_list()
eit_cell.grade.value_counts()

A    999
B     11
Name: grade, dtype: int64

In [17]:
import os

all_files = os.listdir("./8 cell embryo/")
set(eit_cell_images) - set(all_files)

set()

In [18]:
eit_cell_images_new = ["8_cell_" + file for file in eit_cell_images]

In [19]:
for current_name, new_name in zip(eit_cell_images, eit_cell_images_new):
    os.rename(f"./8 cell embryo/{current_name}", f"./8 cell embryo/{new_name}")

In [49]:
eit_cell['image'] = eit_cell_images_new

In [20]:
blasto = pd.read_csv("blastocyte_labels.csv")
blasto.head(2)

image                                         raw_output grade  \
0  00997.png  <developmental_stage>\nThe embryo appears to b...     B   
1  00976.png  <developmental_stage>\nThe embryo appears to b...     A   

        stage  
0  blastocyte  
1  blastocyte

In [21]:
blasto_images = blasto.image.to_list()
blasto.grade.value_counts()

B    716
A    266
C      8
Name: grade, dtype: int64

In [23]:
import os

all_files = os.listdir("./blastocyte/")
set(blasto_images) - set(all_files)

set()

In [24]:
blasto_images_new = ["blastocyte_" + file for file in blasto_images]

In [25]:
for current_name, new_name in zip(blasto_images, blasto_images_new):
    os.rename(f"./blastocyte/{current_name}", f"./blastocyte/{new_name}")

In [50]:
blasto['image'] = blasto_images_new

In [51]:
all_df = pd.concat([morula, blasto, eit_cell])

In [52]:
all_df = all_df[['image', 'grade', 'stage']]

In [53]:
all_df

image grade   stage
0      morula00997.png     C  morula
1      morula00977.png     A  morula
2      morula00975.png     B  morula
3      morula00966.png     B  morula
4      morula00988.png     B  morula
...                ...   ...     ...
1005  8_cell_00031.png     A  8_cell
1006  8_cell_00059.png     A  8_cell
1007  8_cell_00004.png     A  8_cell
1008  8_cell_00001.png     A  8_cell
1009  8_cell_00010.png     A  8_cell

[3014 rows x 3 columns]

In [54]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(all_df, test_size=0.1, stratify=all_df[['grade', 'stage']])

In [63]:
train_data

image grade       stage
884  blastocyte_00147.png     B  blastocyte
600       morula00379.png     A      morula
126  blastocyte_00864.png     B  blastocyte
898       morula00166.png     A      morula
686  blastocyte_00299.png     B  blastocyte
..                    ...   ...         ...
256       morula00802.png     B      morula
470  blastocyte_00510.png     A  blastocyte
111  blastocyte_00846.png     B  blastocyte
32        morula00949.png     A      morula
782       morula00296.png     C      morula

[2712 rows x 3 columns]

In [56]:
train_data.grade.value_counts()

B    1344
A    1286
C      82
Name: grade, dtype: int64

In [57]:
train_data.stage.value_counts()

morula        913
8_cell        909
blastocyte    890
Name: stage, dtype: int64

In [58]:
train_data.to_excel("train_data.xlsx", index=False)

In [59]:
test_data.head(2)

image grade       stage
971  blastocyte_00061.png     A  blastocyte
237  blastocyte_00718.png     B  blastocyte

In [60]:
test_data.grade.value_counts()

B    150
A    143
C      9
Name: grade, dtype: int64

In [61]:
test_data.stage.value_counts()

8_cell        101
morula        101
blastocyte    100
Name: stage, dtype: int64

In [62]:
test_data.to_excel("test_data.xlsx", index=False)

In [ ]:
image_folder =  "./all_images/"

In [68]:
train_data.shape

(2712, 3)

In [86]:
import os
import random
import numpy as np
import pandas as pd
from PIL import Image
import torchvision.transforms as transforms

# Custom function to add Gaussian noise to a PIL image.
def add_gaussian_noise(img):
    np_img = np.array(img)
    # Adjust noise level (here sigma=25); feel free to tune.
    noise = np.random.randn(*np_img.shape) * 25  
    np_img = np_img.astype(np.float32) + noise
    np_img = np.clip(np_img, 0, 255).astype(np.uint8)
    return Image.fromarray(np_img)

def augment_image(image, num_augmented=6):
    """
    Generates a list of augmented images using a random pipeline.
    Each step is applied based on a random decision so that each image
    receives a different combination of transformations.
    
    Parameters:
        image (PIL.Image): The input image.
        num_augmented (int): Number of augmented images to generate.
        
    Returns:
        List[PIL.Image]: Augmented images.
    """
    augmented_images = []
    
    for _ in range(num_augmented):
        transform_list = []
        
        # Random Horizontal Flip (50% chance)
        if random.random() < 0.5:
            transform_list.append(transforms.RandomHorizontalFlip(p=1.0))
        
        # Random Rotation (angle between -15 and 15 degrees, 50% chance)
        if random.random() < 0.5:
            angle = random.uniform(-15, 15)
            transform_list.append(transforms.RandomRotation(degrees=(angle, angle)))
        
        # Random Color Jitter (brightness and contrast, 50% chance)
        if random.random() < 0.5:
            transform_list.append(transforms.ColorJitter(brightness=0.5, contrast=0.5))
        
        # Random Gaussian Blur (50% chance)
        if random.random() < 0.5:
            # Choose a kernel size randomly from odd numbers
            kernel_size = random.choice([3, 5, 7])
            transform_list.append(transforms.GaussianBlur(kernel_size=kernel_size))
        
        # Random Affine Translation (50% chance)
        if random.random() < 0.5:
            # Translate up to 10% of image dimensions
            transform_list.append(transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)))
        
        # Random Gaussian Noise using a Lambda transform (50% chance)
        if random.random() < 0.5:
            transform_list.append(transforms.Lambda(lambda img: add_gaussian_noise(img)))
        
        # In case no transform was applied, add an identity transform
        if not transform_list:
            transform_list.append(transforms.Lambda(lambda img: img))
        
        # Compose the selected transformations into a pipeline
        pipeline = transforms.Compose(transform_list)
        aug_img = pipeline(image)
        augmented_images.append(aug_img)
    
    return augmented_images

# Assume you have a DataFrame 'df' with columns: 'image', 'grade', 'stage'
# And all images are stored in the folder specified by 'image_path'
image_path = "./all_images/"  # Update with your actual path
# For example: df = pd.read_csv("your_dataframe.csv")

# List to store new rows corresponding to augmented images
augmented_rows = []

# Iterate over each row in the dataframe
for idx, row in train_data.iterrows():
    if row['grade'] == "C":
        original_img_name = row['image']
        full_img_path = os.path.join(image_path, original_img_name)
        
        try:
            # Open the image and ensure it's in RGB mode
            img = Image.open(full_img_path).convert('RGB')
        except Exception as e:
            print(f"Could not open image {full_img_path}: {e}")
            continue
        
        # Generate a set number of augmented images using the pipeline
        aug_images = augment_image(img, num_augmented=6)
        
        # Save each augmented image and create a new dataframe row for it
        for i, aug_img in enumerate(aug_images, start=1):
            base_name, ext = os.path.splitext(original_img_name)
            new_filename = f"{base_name}_aug{i}{ext}"
            new_img_path = os.path.join(image_path, new_filename)
            
            try:
                aug_img.save(new_img_path)
            except Exception as e:
                print(f"Error saving image {new_img_path}: {e}")
                continue
            
            # Create a new row with the same 'grade' and 'stage' labels
            new_row = {
                'image': new_filename,
                'grade': row['grade'],
                'stage': row['stage']
            }
            augmented_rows.append(new_row)


In [87]:
len(augmented_rows)

492

In [88]:

# Create a DataFrame from the new augmented rows and append it to the original df
augmented_df = pd.DataFrame(augmented_rows)
df_final = pd.concat([train_data, augmented_df], ignore_index=True)


In [89]:
df_final.head(2)

image grade       stage
0  blastocyte_00147.png     B  blastocyte
1       morula00379.png     A      morula

In [90]:
df_final.shape

(3204, 3)

In [91]:
df_final.grade.value_counts()

B    1344
A    1286
C     574
Name: grade, dtype: int64

In [92]:
df_final.to_excel("train_data_augmented.xlsx", index=False)

In [93]:
df_final.shape

(3204, 3)